In [ ]:
import torch
from torchvision.utils import save_image
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models

from PIL import Image



class vgg_model(nn.Module):
    def __init__(self):
        super(vgg_model, self).__init__()
        self.vgg = models.vgg19(pretrained=True).features[:29]

        self.chosen_layers = ["0", "5", "10", "19", "28"]


    def forward(self, x):
        features = []
        for layer_num, layer in enumerate(self.vgg):
            x = layer(x)

            if str(layer_num) in self.chosen_layers:
                features.append(x)

        return features


def image(name):
    img = Image.open(name)
    img = loader(img).unsqueeze(0)
    return img.to(device)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
size = 256
loader = transforms.Compose(
    [
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

o_img = image("img.jpg")
s_img = image("img2.jpg")
gen = original_img.clone().requires_grad_(True)
model = vgg_model().to(device).eval()

steps = 6000
learning_rate = 0.001
alpha = 1
beta = 0.01
optim = optim.Adam([generated], lr=learning_rate)

for step in range(steps):
    generated_features = model(gen)
    original_img_features = model(o_img)
    style_features = model(s_img)
    s_loss = o_loss = 0
    for gen_feature, orig_feature, style_feature in zip(
        generated_features, original_img_features, style_features
    ):
        batch_size, channel, height, width = gen_feature.shape
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)
        G = gen_feature.view(channel, height * width).mm(
            gen_feature.view(channel, height * width).t()
        )
        A = style_feature.view(channel, height * width).mm(
            style_feature.view(channel, height * width).t()
        )
        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    optim.zero_grad()
    total_loss.backward()
    optim.step()

    if step % 50 == 0:
        print(total_loss)
        save_image(gen, "generated.png")